## Het Plan
Voor mijn verdiepende opdracht wil ik GPT2 trainen op oncologische teksten. De data zal van Wikipedia en NCBI m.b.v Entrez Programming Utilities komen. Misschien worden er statistieken van SEER (statistieken in USA) gebruikt. Een extra doel is dat het model vragen kan beantwoorden. zolang dat niet geimplementeerd is wordt de input als seed gebruikt bv. 'Colon Cancer is'.

## Dataverzameling
Om te beginnen is er data nodig, hoe meer hoe beter. Ik begon meteen met Entrez, hier is goede [documentatie](https://biopython.org/docs/1.76/api/Bio.Entrez.html) van, met duidelijke voorbeelden. Ik zoek eerst naar id's gerelateerd aan `cancer`, en haal daarna het artikel op met efetch (m.b.v. het id). De artikelen zijn in `xml` formaat. De efetch kan ook in 'gewone' `text` (tekst) de artikelen geven, maar om een reden waar ik niet achter ben gekomen krijg ik dan alleen de abstract.

Ik maak gebruik van een API-key om zo van 3 requests/seconde naar 10 requests/seconde te gaan. Na wat succesvolle testen probeerde ik 1000 artikelen op te vragen:

In [ ]:
from Bio import Entrez
import time

def uidfinder():
    Entrez.email = 'bram.koobs11@gmail.com'
    Entrez.api_key = '1e3618cf8085ceaaa4cec35608f05ae72d09'
    search_handle = Entrez.esearch(db="pmc", term="cancer", retmax=1000)
    search_results = Entrez.read(search_handle)
    uids = search_results['IdList']
    return uids


def uidfetcher(uids):
    Entrez.email = 'bram.koobs11@gmail.com'
    Entrez.api_key = '1e3618cf8085ceaaa4cec35608f05ae72d09'
    for uid in uids:
        fetch_handle = Entrez.efetch(db='pmc', id=uid, rettype='full', retmode='xml')
        record = fetch_handle.read()
        
        with open(f'data/{uid}.xml', 'wb') as file:
            file.write(record)
        time.sleep(0.2) #wait 200ms between calls, the key allows for max 10 request/sec (this does 5/s)

uids = uidfinder()
#print(uids)
uidfetcher(uids)

Artikelen krijgen is redelijk gelukt, ik heb nu ~100 artikelen. Ik verwachte zonder problemen zoveel artikelen te kunnen opvragen als ik wou. Maar na 30-60 seconden krijg ik een HTTPError:
> HTTP Error 400: Bad Request

Aanvankelijk ging ik ervan uit dat ik teveel request per seconde deed, dus in plaats van 10/s had ik het verlaagd naar 5/s. Dit loste het probleem niet op. Na wat zoeken online vond ik dat sommigen het konden oplossen door een andere `db` (database) te kiezen. Mij lijkt pubmed de beste database voor artikelen vinden, dus dit leek mij niet een geschikte oplossing. Omdat de HTTPError niet meteen opkomt, ligt het hoogswaarschijnlijk aan individuele artikelen. Als oplossing heb ik de efetch in een `try` block gezet:

In [7]:
def uidfetcher(uids):
    Entrez.email = 'bram.koobs11@gmail.com'
    Entrez.api_key = '1e3618cf8085ceaaa4cec35608f05ae72d09'
    for uid in uids:
        try:
            fetch_handle = Entrez.efetch(db='pmc', id=uid, rettype='full', retmode='xml')
            record = fetch_handle.read()
        except:
            print(uid)
        
        with open(f'data/{uid}.xml', 'wb') as file:
            file.write(record)
        time.sleep(0.15) #wait 150ms between calls, the key allows for max 10 request/sec

uidfetcher(uids)

11756136
11755543
11755542
11755530
11755156
11755021
11754228
11753914
11753461
11753077
11751778
11750679
11750677
11750631
11750132
11750099


Na 2 minuten werd de eerste uid geprint. Dit [artikel](https://pubmed.ncbi.nlm.nih.gov/11756136/) heeft (op pubmed) geen inhoud, wat waarschijnlijk voor de problemen zorgt.

Samen met de eerdere test-runs heb ik nu precies 1000 artikelen.

## Dataverwerking
Alle artikelen zijn in xml-formaat, wat nog omgezet moet worden naar tekst. Om te beginnen wordt de body (alles wat tussen `<body>` en `</body>` in zit) uit het artikel gehaald. Waarna alle tags worden verwijderd met re.sub. Tot slot worden regels die beginnen met een `\` weggelaten.


Lang niet alle artikelen die te vinden zijn op pubmed hebben het hele artikel op pubmed zelf staan. De manier die wordt gebruikt om te beslissen of het artikel inhoud heeft is door te kijken het vinden van tekst tussen `<body>` en `</body>` lukt. Als dit niet lukt, en deze tags dus niet voorkomen wordt het artikel niet verder gebruikt.

In [2]:
import os
import re

dir_list = os.listdir('data/')

for file_name in dir_list:
    file_name = file_name.split('.')[0] #extract uid from file name
    with open(f'data/{file_name}.xml', 'r') as file:
        content = file.read()
    #print(file_name, " \n")

    try: #extract body if body exists, if not -> go to next file_name
        body = re.search(r'<body>(.*?)</body>', content, re.DOTALL).group(1)
    except:
        continue
    
    #Remove all XML tags
    text = re.sub(r'<[^>]+>', '', body)
    #filter out lines starting with a backslash, an '&' and/or less than 6 chars
    filtered_text = '\n'.join([line for line in text.splitlines() 
    if not line.strip().startswith('\\') and len(line.strip()) > 5
    and not line.strip().startswith('&')])

    #filtered_text = '\n'.join([line for line in text.splitlines() if not line.strip().startswith('\\')])

    with open(f'text/{file_name}.txt', 'w') as f:
        f.write(filtered_text)


Na het verwerken van de artikelen zijn er van de 1000, 886 over.

In [1]:
import os
from datasets import load_dataset
from datasets import DatasetDict

files = os.listdir('text/')
val_file_amount = int(len(files)/5)

def merger(files): #merges multiple text-files into 1 text-file
    output = ''
    for filename in files:
        with open(f'text/{filename}', 'r', encoding='utf-8') as infile:
            output += infile.read() + '\n'
    return output

validation_lines = merger(files[0:val_file_amount])
train_lines = merger(files[val_file_amount:])

with open('train_data.txt', 'w', encoding='utf-8') as f:
    f.writelines(train_lines)

with open('validation_data.txt', 'w', encoding='utf-8') as f:
    f.writelines(validation_lines)

dataset = load_dataset('text', data_files={'train': 'train_data.txt', 'validation': 'validation_data.txt'})


def remove_empty_text(dataset):
    return dataset.filter(lambda example: example['text'].strip() != '')

#Apply the filter to both train and validation sets
filtered_dataset = DatasetDict({
    split: remove_empty_text(dataset[split]) for split in ["train", "validation"]
})

#Verify the structure
print(filtered_dataset)

#Train a new tokenizer using the training data
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

#Save the tokenizer
tokenizer.save_pretrained("tokenizer")

d:\HW\Jaar3\Blok3\VerdiependModel\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 93086 examples [00:00, 387556.09 examples/s]
Generating validation split: 24530 examples [00:00, 347827.32 examples/s]
Filter: 100%|██████████| 24530/24530 [00:00<00:00, 331413.34 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 93086
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 24530
    })
})


('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.json',
 'tokenizer\\merges.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [2]:
from transformers import GPT2Tokenizer

#Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

#Add a padding token (GPT-2 doesn't have one by default)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

# Apply the tokenization function to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Format the dataset for training (convert it to PyTorch tensors)
tokenized_dataset.set_format("torch")

print(tokenized_dataset)

Map: 100%|██████████| 24530/24530 [00:15<00:00, 1634.86 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 93086
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 24530
    })
})



databases
transformers
pytorch
transformers[torch]
ipykernel

In [3]:
from transformers import GPT2Config, GPT2LMHeadModel

# Initialize a GPT-2 configuration
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=1024,
    n_ctx=1024,
    n_embd=768,
    n_layer=12,
    n_head=12,
)

# Create a new GPT-2 model
model = GPT2LMHeadModel(config)


## Trainen

Bij het trainen van het model liep ik aanvankelijk tegen dit probleem aan: \
`RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with 'TORCH_USE_CUDA_DSA' to enable device-side assertions.`

Ik heb de suggesties proberen te implementeren op deze manier:
> import os \
> os.environ['CUDA_LAUNCH_BLOCKING']="1" \
> os.environ['TORCH_USE_CUDA_DSA'] = "1"

Dit veroorzaakte geen merkbaar verschil. Vervolgens heb ik de `Nvidia CUDA toolkit` geinstalleerd. Deze toolkit is niet nodig voor pytorch en tensorflow, maar voor Huggingface blijkt dit wel nodig te zijn. Voor de zekerheid heb ik met `pip install cuda-python` cuda-python geinstalleerd.

Het opnieuw runnen van de code produceerd nu deze error: \
`RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with 'TORCH_USE_CUDA_DSA' to enable device-side assertions.`

Met behulp van nvidia-smi in de command prompt heb ik (current) memory usages bekeken, deze stond op 633MiB / 24576MiB, wat ruim genoeg is. Om er zeker van te zijn dat de memory niet echt 'op' raakt heb ik deze test gedaan:


In [4]:
import torch
foo = torch.tensor([1,2,3])
foo = foo.to('cuda')

AssertionError: Torch not compiled with CUDA enabled

~24gb aan memory zou deze matrix aan moeten kunnen. Om te kijken of CUDA überhaupt wel beschikbaar is run ik de volgende tests:

In [8]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

False
None


Cuda is `available`, maar is versie `12.1`. nvidia-smi geeft aan dat mijn GPU CUDA versie 12.8 gebruikt. Na wat zoeken online lijkt dit niet een probleem te zijn.

Om er zeker van te zijn dat alle problemen niet door incompatibility worden veroorzaakt. heb ik een venv aangemaakt.

In [58]:
print(f"Vocabulary size: {tokenizer.vocab_size}")

Vocabulary size: 50257


In [7]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

# Define a data collator that handles padding dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="gpt2_from_scratch",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="logs",
    logging_steps=100,
    learning_rate=5e-4,
    weight_decay=0.01,
    warmup_steps=500,
)


# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the trained model and tokenizer
model.save_pretrained("trained_gpt2")
tokenizer.save_pretrained("trained_gpt2")


d:\HW\Jaar3\Blok3\VerdiependModel\venv\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


IndexError: index out of range in self